Loading Data using CSV

In [1]:
# prompt: load data from google drive

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Replace 'your_file_path.csv' with the actual path to your CSV file in Google Drive
df = pd.read_csv('/content/drive/MyDrive/cleaned_data.csv')

print(df.head())

Mounted at /content/drive
   productID           ram      harddisk    price  model_ACER 14 INCHES  \
0          0  4.000000e+09  5.000000e+11   180.00                 False   
1          1  1.600000e+10  5.120000e+11  1211.99                 False   
2          2  3.200000e+10  2.000000e+12   735.59                 False   
3          3  4.000000e+09  5.120000e+11   339.00                 False   
4          4  1.200000e+10  1.000000e+12  2086.99                 False   

   model_ACER 15.6 INCHES  model_ACER 17.3 INCHES  \
0                   False                   False   
1                   False                   False   
2                   False                   False   
3                   False                   False   
4                   False                   False   

   model_ACER 2023 NEWEST ACER 15.6" CHROMEBOOK 15.6 INCHES  \
0                                              False          
1                                              False          
2              

In [ ]:
import pandas as pd
df = pd.read_csv('cleaned_data.csv')

Splitting the Data:

In [2]:
from sklearn.model_selection import train_test_split
X = df.drop('price', axis=1)
y = df['price']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


Choosing a Model:

Linear Regression:

In [3]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

Random Forest:



In [6]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

Model Evaluation: After training the model, evaluate its performance using RMSE.



In [10]:
import numpy as np
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print('Validation RMSE:', rmse)


Validation RMSE: 610.5622020407242


Hyperparameter Tuning:

If the RMSE is not below 300, you can tune the model’s hyperparameters using GridSearchCV.


In [8]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_


Testing

Load the Test Data:



In [12]:
test_df = pd.read_csv('test.csv')
test_df_cleaned = preprocess(test_df)  # Use the same preprocessing steps as the training data


Make Predictions:

In [14]:
predictions = best_model.predict(test_df)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- OS
- color
- cpu
- gpu
- model
- ...
Feature names seen at fit time, yet now missing:
- OS_Chrome OS
- OS_HP ThinPro
- OS_LINUX
- OS_MAC OS
- OS_MACOS 10.14 MOJAVE
- ...


Prepare Submission:



In [ ]:
submission = pd.DataFrame({'productID': test_df['productID'], 'PredictedPrice': predictions})
submission.to_csv('submission.csv', index=False)


Insights and Summary
Summarize the important findings in both notebooks, discussing:

Which features had the most significant impact on price
The final RMSE on the test set
Any insights into how certain components (e.g., RAM, CPU, GPU) affect the price of notebooks.
